Installing the necessary packages also make sure that you are connected to T4 GPU or any other GPU runtime in collab

In [1]:
! pip install gradio transformers torch tiktoken verovio easyocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 86.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 102.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 89.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 116.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2

The application utilizes GOT OCR along with EasyOCR and also highlights the given keyword. User just needs to upload any image of their choice and the keyword and will get responses from both EasyOCR and GOT ( 580M end-to-end OCR 2.0 model). Right now EasyOCR gives best result for Hindi text and GOT for english texts so utilized both the services.

In [2]:
import gradio as gr
from transformers import AutoTokenizer, AutoModel
import torch
import easyocr
from PIL import Image

def load_got_model():
    tokenizer = AutoTokenizer.from_pretrained('ucaslcl/GOT-OCR2_0', trust_remote_code=True)
    model = AutoModel.from_pretrained('ucaslcl/GOT-OCR2_0', trust_remote_code=True, low_cpu_mem_usage=True, device_map='cuda', use_safetensors=True, pad_token_id=tokenizer.eos_token_id)
    return tokenizer, model.eval().cuda()

def load_easyocr():
    reader = easyocr.Reader(['en', 'hi'])
    return reader

tokenizer, got_model = load_got_model()
easyocr_reader = load_easyocr()

def perform_ocr(image, keyword):
    image.save("temp_image.png")

    got_res = got_model.chat(tokenizer, "temp_image.png", ocr_type='ocr')

    easyocr_res = easyocr_reader.readtext("temp_image.png", detail=0)
    easyocr_res = ' '.join(easyocr_res)

    highlight_color = "#87CEEB"

    if keyword:
        got_highlighted = got_res.replace(keyword, f"<mark style='background-color:{highlight_color};'>{keyword}</mark>")
        easyocr_highlighted = easyocr_res.replace(keyword, f"<mark style='background-color:{highlight_color};'>{keyword}</mark>")
        return got_highlighted, easyocr_highlighted

    return got_res, easyocr_res

interface = gr.Interface(
    fn=perform_ocr,
    inputs=[
        gr.Image(type="pil", label="Upload Image"),
        gr.Textbox(label="Keyword to Search", placeholder="Enter keyword to highlight")
    ],
    outputs=[
        gr.HTML(label="GOT OCR Result"),
        gr.HTML(label="EasyOCR Result")
    ],
    title="OCR Application using GOT 2.0 and EasyOCR",
    description="Upload an image to extract text using the GOT OCR 2.0 model and EasyOCR (English and Hindi), and search for keywords in the extracted text."
)

if __name__ == "__main__":
    interface.launch()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/300 [00:00<?, ?B/s]

tokenization_qwen.py:   0%|          | 0.00/9.47k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ucaslcl/GOT-OCR2_0:
- tokenization_qwen.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


qwen.tiktoken:   0%|          | 0.00/2.56M [00:00<?, ?B/s]

...

A new version of the following files was downloaded from https://huggingface.co/ucaslcl/GOT-OCR2_0:
- render_tools.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/ucaslcl/GOT-OCR2_0:
- modeling_GOT.py
- got_vision_b.py
- render_tools.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% CompleteSetting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://309056624da9aa8c04.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
